## Create Test Data
Just a quick script to load in a data file, reduce memory by selecting 1 time value, less lat and lon and save as output.

In [1]:
import os
import xarray as xr
import time
import numpy as np

In [53]:
lat_ind_keep = np.linspace(0, 192-1, 5, dtype=int)
lon_ind_keep = np.linspace(0, 288-1, 10, dtype=int)
time_ind_keep = [4, 5]

def preprocess(ds):
    return ds.isel(lat=lat_ind_keep, lon=lon_ind_keep, time=time_ind_keep)
#
# def preprocess(ds):
#     return ds['SOILLIQ']
#
# def preprocess(ds):
#     return ds['T']

In [54]:
dir_desktop = os.environ['HOME'] + '/Desktop/'
exp_name_land = 'e.e20.E1850TEST.f09_g17.daily_output.clm2.h1.0031-01-02-00000.nc'
exp_name_atm = 'e.e20.E1850TEST.f09_g17.daily_output.cam.h1.0031-01-02-00000.nc'

In [55]:
def load_with_profile(file_path, preprocess=preprocess, process_first=True, chunks=None, chunk_first=True):
    time_info = []
    time_info += [time.time()]
    ds = xr.open_mfdataset(file_path, preprocess=preprocess if process_first else None, chunks=chunks if chunk_first else None)
    time_info += [time.time()]
    print("Lazy loading: {:.3f}s".format(time_info[-1] - time_info[-2]))
    if not chunk_first and chunks is not None:
        ds = ds.chunk(chunks)
        time_info += [time.time()]
        print("Chunking: {:.3f}s".format(time_info[-1] - time_info[-2]))
    if not process_first:
        ds = preprocess(ds)
        time_info += [time.time()]
        print("Processing: {:.3f}s".format(time_info[-1] - time_info[-2]))
    ds.load()
    time_info += [time.time()]
    print("Load in full: {:.3f}s".format(time_info[-1] - time_info[-2]))
    print("Total time: {:.3f}s".format(time_info[-1] - time_info[0]))
    return ds

In [56]:
ds_land = load_with_profile(dir_desktop + exp_name_land)

Lazy loading: 0.053s
Load in full: 3.442s
Total time: 3.495s


In [57]:
ds_atm = load_with_profile(dir_desktop + exp_name_atm)

Lazy loading: 0.059s
Load in full: 27.488s
Total time: 27.547s


In [58]:
ds_atm.to_netcdf(dir_desktop + f'atm_time={time_ind_keep}.nc')
ds_land.to_netcdf(dir_desktop + f'land_time={time_ind_keep}.nc')